A Schulz style treatment of conditionals, for a revised approach to earlier work in this directory. The key innovation/value is separating the dynamics from the possible worlds in a way that makes things easier to reason about. 


key issues to resolve: how do presuppositions and conditional presuppositions end up working out? how do local contexts work out?

how do we (efficiently) calculate the basis of a world given a dynamics?



We can simplify our models by getting rid of the U variables. 



# Key sections:
- Define a class for our dynamics. 
    - Should provide a set of helper functions for determining the truth of an expression 
    - definition of causal dependence. 
    - definition of 

- Define an interpretation function. 

- Define objects for conversational structures. 






In [ ]:
# For handling trivalent logic with strong Kleene
from trinary import Unknown, strictly, weakly
from copy import copy, deepcopy
from itertools import chain, combinations
import numpy as np
from functools import lru_cache

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def contains(l1, l2):
    return all ([i in l1 for i in l2])

def complement(u, l):
    return [i for i in u if i not in l]

def all3val(l):

    value = True

    i = 0
    
    while (i < len(l)):
        value = value & l[i]
        i += 1

    return value

def any3val(l):

    value = False

    i = 0

    while (i < len(l)):
        value = value | l[i]
        i += 1

    return value

def ident(x):
    return x

class State: 

    def __init__(self, universe, tset, fset):
        """
        universe: universe of atomic propositions. 
        tset: set of true atomic propositions. 
        fset: set of false atomic propositions. 
        """

        self._universe = set(universe)
        self._tset = set(tset)
        self._fset = set(fset)

        # TODO: Add checks to ensure consistent state definitions.

    def __getitem__(self, proposition):

        if proposition not in self._universe:
            return "Error"

        if proposition in self._tset: 
            return True 
        elif proposition in self._fset:
            return False
        else:
            return  Unknown
        
    def __len__(self):
        return len(self._tset.union(self._fset))
        
    def __eq__(self, other):
        return self._universe == other._universe and self._tset == other._tset and self._fset == other._fset
    
    def __lt__(self, other):
        return self._universe == other._universe and len(self) < len(other) and contains(other._tset, self._tset) and contains(other._fset, self._fset) and contains(self.undefined(), other.undefined())

    def __le__(self, other):
        return self._universe == other._universe and len(self) <= len(other) and contains(other._tset, self._tset) and contains(other._fset, self._fset) and contains(self.undefined(), other.undefined())

    def __hash__(self):
        pass

    def defined(self):
        return self._tset.union(self._fset)

    def undefined(self):
        return self._universe - self.defined()
        
    def remap(self, proposition, value):

        if strictly(self[proposition]): 
            self._tset.remove(proposition)
        elif not (weakly(self[proposition])):
            self._fset.remove(proposition)
        
        if strictly(value):
            self._tset.add(proposition)
        elif not (weakly(value)):
            self._fset.add(proposition)

    

        

In [546]:
# A Schulz style treatment of conditionals, for a revised approach. 

class Dynamics:
    
    def __init__(self, background, maps):

        self._background = set(background)
        self._maps = maps

    def __getitem__(self, p):
        """
        Given an internal variable, returns its value based upon the 
        """
        return self._maps[p]
    
    def get_background(self):
        return self._background

    def getvalue(self, p, s):

        if p in self._maps.keys():

            f, parents = self._maps[p]

            return f ([s[par]for par in parents])
        
        else:
            return s[p]


    def run_model(self, s):
        """
        situation: mapping of propositions to values.
        returns: revised state
        """

        # copy our starting situation
        s_new = deepcopy(s)

        internal_variables = [v for v in s._universe if v not in self._background]

        for i in internal_variables:
            value = self.getvalue(i, s)
            if strictly(s[i]) != weakly(s[i]) and strictly(value) == weakly(value):
                s_new.remap(i, value)
        
        return s_new

    def fixed_point(self, s):

        s_new = self.run_model(s)

        while (s_new != s):
            s = deepcopy(s_new)
            s_new = self.run_model(s)

        return s_new

    def find_basis(self, w):

        local_universe = w._universe

        tsubsets = [set(i) for i in list(powerset(w._tset))]
        fsubsets = [set(i) for i in list(powerset(w._fset))]

        # find all subset situations, filter those for ones that meet the basis condition and then take the smallest. 

        subsituations = [s for t in tsubsets for f in fsubsets for s in [State(local_universe, t, f)] if self.fixed_point(s) == w]

        return min(subsituations)
    

        

        

        





In [547]:

def parse_to_tree(message):
    # Maybe eventually replace this with a more general parsing algorithm

    connectives = [">", "-", "d"]

    if not any (conn in message for conn in connectives):
        return ["leaf", message]
    
    else: 
        depths = []
        connective_ixs = []
        depth = 0

        for i, c in enumerate(message):
            if c == '(':
                depth += 1
            elif c == ')':
                depth -= 1
            elif c in connectives:
                depths.append(depth)
                connective_ixs.append(i)

        least_c_ix = connective_ixs[np.argmin(depths)]

        connective = message[least_c_ix]

        if connective == '-':
            if np.min(depths) == 0:
                return [connective, parse_to_tree(message[least_c_ix + 1:])]
            else:
                return [connective, parse_to_tree(message[least_c_ix + 1:-1])]

        if np.min(depths) == 0:
            return [message[least_c_ix], parse_to_tree(message[:least_c_ix]), parse_to_tree(message[least_c_ix + 1:])]
        else: 
            return [message[least_c_ix], parse_to_tree(message[1:least_c_ix]), parse_to_tree(message[least_c_ix + 1:-1])]



         
            


The next problem I want think about is how we can arrive at a set of worlds which is causally consistent with a dynamics - there are going to be a lot of strange sets of possible worlds that we might consider which could fuck with the basis and the definition for conditionals that we've arrived at. I should try to sharpen this intuition. 

We can imagine an example - imagine basically any world and dynamics where the basis is not the set of external variables in the model. For the switch example, one is given below:

In [ ]:
switch_universe = ["a", "b", "c"]

w3 = State(["a", "b", "c"], ["a", "c"], ["b"])



On this kind of world, our treatment of the conditional will break a little bit. The solution, I suspect, will be to model inference of an expression, sensitive to multiple possible dynamics. 

To begin setting this up, we first need to think a little more seriously about how we want to manage our intepretation. We can define the interpretation function relative to a state. 

We also need to think about how we're going to manage presuppositions. The approach that I can see is to embed a failure value - so that a presupposition, evaluated against a world where it isn't true, returns the unknown value, and we define a bridge principle that requires that the expression be well-defined throughout the context. 

To manage conditionals, we'll define a version of causal entailment that is sensitive to presupposition. We'll start by redefing our interpretation function to handle our connectives, new literals, and whatever else:

In [636]:

def interpret(tree, world, dynamics):
    root_node_label = tree[0]

    if root_node_label == "leaf": 
        proposition = tree[1]
        return leaf(proposition, world)
    elif root_node_label == "d":
        assertion = tree[1]
        presupposition = tree[2]
        return presupposes(assertion, presupposition, world, dynamics)
    elif root_node_label == "-":
        prejacent = tree[1]
        return neg(prejacent, world, dynamics)
    elif root_node_label == ">":
        antecedent = tree[1]
        consequent = tree[2]
        return conditional(antecedent, consequent, world, dynamics)


def leaf(p, world):
    return world[p]


def presupposes(tree1, tree2, world, dynamics):
    # tree1 presupposes tree2
    if not interpret(tree2, world, dynamics):
        return Unknown
    else:
        return interpret(tree1, world, dynamics)

def neg(tree, world, dynamics):
    return ~ interpret(tree, world, dynamics) # use bitwise negation to handle the Unknown value


def consequence(tree, situation, dynamics):
    fixed_point = dynamics.fixed_point(situation)
    return interpret(tree, fixed_point, dynamics)

def conditional(tree1, tree2, world, dynamics):
    
    p, val = literal(tree1)
    
    basis = dynamics.find_basis(world)
    basis.remap(p, val)

    return consequence(tree2, basis, dynamics)


def literal(tree1):

    if tree1[0] == "leaf":
        return (tree1[1], True)
    elif tree1[0] == "-":
        return (tree1[1][1], False)

In [551]:
parse_to_tree("-(a>b)>c")
# things are right associative because min defaults to the first minimum - this is cool to me

['-', ['>', ['>', ['leaf', 'a'], ['leaf', 'b']], ['leaf', 'c']]]

In [639]:
# this will be our basic evaluation function for the literal listener, requires that the message be true at every world in the state, and defined at every world in the background. 

# we'll also later check that the state is consistent with the background and not entailed (proper subset of), and that the background is causally consistent with the dynamics (for each world, its basis corresponds to its external variables)

# we can actually manage state as a single world 

def evaluate(message, world, background, dynamics):
    message_tree = parse_to_tree(message)
    
    defined = all3val ([strictly(interpret(message_tree, b, dynamics)) == weakly(interpret(message_tree, b, dynamics)) for b in background])

    return strictly(interpret(message_tree, world, dynamics)) and defined

def causally_consistent(w, dynamics):
    basis = dynamics.find_basis(w)

    return basis.defined() == dynamics.get_background()


In [554]:
scubaverse = ["s", "w", "o"]

scubynamics1 = Dynamics(["s"], {"w": (all3val, ["s"]),
                                "o": (all3val, ["w"])})

scubynamics2 = Dynamics(["s"], {"w": (all3val, ["s"]),
                                "o": (all3val, ["w", "s"])})

# we should rename this to sitation in the cleaned up version
squid1 = State(scubaverse, ["s", "w", "o"], [])

In [ ]:
causally_consistent(squid1, scubynamics1) 

# to actually make things interesting, we probably need to add additional exogenous variables, an easy way of introducing incomplete dependence. 

True

In [555]:
def log_inf(x):
    return np.log(x) if x > 0 else -float('Inf')

def default_cost(m):
    return len(m)




In [ ]:


class RSA:
    
    def __init__(self, messages, states, backgrounds, dynamics, interpretation_function, cost=default_cost, temp=1):

        self.messages           = messages
        self.states             = states
        self.backgrounds        = backgrounds
        self.dynamics           = dynamics
        self.interpretation     = interpretation_function
        self.cost               = cost
        self.temp               = temp


    def literal_listener(self, message, background, dyn):
        # message, background, 

        state_prior = np.array([1 * (s in background and len(background) > 1) for s in self.states])

        background_prior = 1 * all3val([causally_consistent(w, dyn) for w in background])

        probs = np.array([self.interpretation(message, s, background, dyn) for s in self.states]) * state_prior * background_prior

        return probs / sum(probs) if sum(probs) > 0 else probs
    
    
    def pragmatic_speaker(self, state_ix, background, dyn):
        # return distribution over messages. 

        U = np.array([np.vectorize(log_inf)(self.literal_listener(message, background, dyn))[state_ix] for message in self.messages])

        probs = np.exp(self.temp * (U - np.array([self.cost(message) for message in self.messages])))

        return probs / sum(probs) if sum(probs) > 0 else probs
    

    def pragmatic_listener(self, message_ix):


        # 2D : B x S 
        speaker = np.array([[self.pragmatic_speaker(s, b, self.dynamics)[message_ix] for s, _ in enumerate(self.states)] for b in self.backgrounds ])
       
        # 2D : B x S
        state_prior = np.array([[1 * (s in b and len(b) > 1) for s in self.states] for b in self.backgrounds] )

        # 2D : B x S
        background_prior = np.array([[1 * all3val([causally_consistent(w, self.dynamics) for w in b]) for s in self.states] for b in self.backgrounds])

        probs = speaker * state_prior * background_prior

        norm = sum(sum(probs))

        return probs / norm if norm > 0 else probs

    
    # def speaker2(self, state_ix, background, dyn):




        


In [ ]:
# Build a model for evaluation

scubaverse = ["s", "w", "o", "u1", "u2"]

scubynamics1 = Dynamics(["s", "u1", "u2"], {"w": (any3val, ["s", "u1"]),
                                            "o": (any3val, ["w", "u2"])})

scubynamics2 = Dynamics(["s", "u1", "u2"], {"w": (any3val, ["s", "u1"]),
                                            "o": (all3val, ["w", "s", "u2"])})

scubynamics3 = Dynamics(["s", "u1", "u2"], {"w": (all3val, ["u1"]), 
                                            "o": (all3val, ["w", "s", "u2"])})
# a possible modification would be to consider a complex rule for our consequent (o and (s or u2))                                            

# scuba_world_sets = [set(s) for s in powerset(scubaverse)]


def build_model(universe, dynamics, messages,  cost=default_cost):

    scuba_world_sets = [set(s) for s in powerset(universe)]

    scuba_worlds = [s for s1 in scuba_world_sets for s2 in scuba_world_sets if len(s1.intersection(s2)) == 0 and len(s1.union(s2)) == len(universe) for s in [State(s1.union(s2), s1, s2)] if causally_consistent(s, dynamics)]

    scuba_backgrounds = [list(s) for s in powerset(scuba_worlds) if len(s) > 0]



    return RSA(messages, scuba_worlds, scuba_backgrounds, dynamics, evaluate, cost)

# model1 = build_model(scubaverse, scubynamics3)
# 
# model3 = build_model(scubaverse, scubynamics1)
# model3prime = build_model(scubynamics1, cost = lambda x : 1)

In [717]:
# m1_l1 = model1.pragmatic_listener(0)
# m2_l1 = model2.pragmatic_listener(0)
m3_l1 = model3.pragmatic_listener(0)

In [737]:
m3_l1_prime = model3prime.pragmatic_listener(0)

In [700]:

def get_max_indices(listener):

    maxp = listener.max()

    (nb, ns) = listener.shape

    flat = listener.flatten()

    return [(i // ns, i % ns) for i in [ix for (ix, val) in enumerate(flat) if val == maxp]]

# m1_ix = get_max_indices(m1_l1)
# m2_ix = get_max_indices(m2_l1)
m3_ix = get_max_indices(m3_l1)



In [653]:
def check_value(message, background, dynamics):
    return all3val ([evaluate(message, w, background, dynamics) for w in background])

In [ ]:
check_value('s>w', model3.states, scubynamics1)



True

In [749]:
# writing a general interpretation script:

# first what's true at each world, do any presuppostions follow from the causal consistency requirement

# then, which presuppositions hold at which backgrounds... 

def interpret_results(model, listener, target, cp, gp):

    pmax = listener.max()

    print("The max probability was ", pmax)

    ixs = get_max_indices(listener)

    max_backgrounds = [b for b, _ in ixs]
    max_worlds = [w for _, w in ixs]

    states = model.states

    # Sanity checks, we don't necessary expect/want these to hold. 
    if check_value(target, states, model.dynamics):
        print("The utterance is trivial at every causally consistent world in the model.")

    if check_value(cp, states, model.dynamics):
        print("The conditional presupposition holds at every (causally consistent) world in the model.")
    elif any3val ([check_value(cp, [s], model.dynamics) for s in states]):
        print("The conditional presupposition holds in at least one causally consistent world in the model.")

    if check_value(gp, states, model.dynamics):
        print("The global presupposition holds at every causally consistent world in the model.")

    elif any3val ([check_value(gp, [s], model.dynamics) for s in states]):
        print("The global presupposition holds in at least one causally consistent world in the model.")

    # Conditional presupposition
    if all3val([check_value(cp, model.backgrounds[b], model.dynamics) for b in max_backgrounds]):
        print("The conditional presupposition holds in all optimal backgrounds.")

    if all3val([check_value(gp, model.backgrounds[b], model.dynamics) for b in max_backgrounds]):
        print("The global presupposition holds in all optimal backgrounds.")

    elif any3val([check_value(gp, model.backgrounds[b], model.dynamics) for b in max_backgrounds]):
        print("The global presupposition holds in at least one optimal background.")
    
    else:
        print("The global presupposition holds in no optimal backgrounds.")






In [718]:
interpret_results(model3, m3_l1)

# Good result here! 
# The lack of conditional link forces a condtional, but not global presupposition. 
# None of the possible backgrounds entail the global presupposition. 

The max probability was  0.0011734723752953918
The utterance is trivial at every causally consistent world in the model.
The conditional presupposition holds at every (causally consistent) world in the model.
The global presupposition holds in at least one causally consistent world in the model.
The conditional presupposition holds in all optimal backgrounds.
The global presupposition holds in no optimal backgrounds.


(255, 8)

In [704]:
interpret_results(model3prime, m3prime_l1)

The max probability was  0.0021668472372697724
The conditional presupposition holds at every (causally consistent) world in the model.
The global presupposition holds in at least one causally consistent world in the model.
The conditional presupposition holds in all optimal backgrounds.
The global presupposition holds in no optimal backgrounds.


In [ ]:
model3prime = build_model(scubynamics1, cost=lambda x: 1)
m3p_l1 = model3prime.pragmatic_listener(0)

Currently, the system we've set up is not rich enough to distinguish between the cases we want to distinguish - this is because there's only one world which satisfies presuppositions and the properties of the assertion.  

In [ ]:
interpret_results(model3prime, m3p_l1)

# YAY this is a good result. 

The max probability was  0.04981859480793297
The utterance is trivial at every causally consistent world in the model.
The conditional presupposition holds at every (causally consistent) world in the model.
The global presupposition holds in at least one causally consistent world in the model.
The conditional presupposition holds in all optimal backgrounds.
The global presupposition holds in at least one optimal background.


In [ ]:
# literal listeners (obviating the need for the RSA)

m3_l0 = [model3.literal_listener('s>(odw)', b, model3.dynamics) for b in model3.backgrounds]

m3_l0 = np.array(m3_l0)
norm = sum(sum(m3_l0))
m3_l0 = m3_l0 / norm 

In [ ]:
interpret_results(model3, m3_l1)

# 

The max probability was  0.0011734723752953918
The utterance is trivial at every causally consistent world in the model.
The conditional presupposition holds at every (causally consistent) world in the model.
The global presupposition holds in at least one causally consistent world in the model.
The conditional presupposition holds in all optimal backgrounds.
The global presupposition holds in no optimal backgrounds.


In [746]:
# EXPERIMENT 1

universe1 = ["a", "b", "c"]

dynamics1 = Dynamics(["a", "b"], 
                     {"c": (all3val, ["a", "b"])})

messages = ["a>(cdb)", "cdb"]

model = build_model(universe1, dynamics1, messages)

l0 = np.array([model.literal_listener("a>(cdb)", b, model.dynamics) for b in model.backgrounds])


In [750]:
interpret_results(model, l0, "a>(cdb)", "a>b", "b")

The max probability was  0.5
The conditional presupposition holds in at least one causally consistent world in the model.
The global presupposition holds in at least one causally consistent world in the model.
The conditional presupposition holds in all optimal backgrounds.
The global presupposition holds in all optimal backgrounds.


# Experiment 3 : Deeper recursions? Worth it?

s